1. Import Libraries

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge geocoder --yes
import geocoder # to get coordinates

#!conda install -c conda-forge bs4 --yes
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


2. Get Dataset

In [7]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Mumbai").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html5lib')

In [8]:
# create a list to store neighborhood data
TouristPlaceList = []
# append the data into the list
#for row in soup.find_all("div",class_="toc")[0].findAll("li"):
#    TouristPlaceList.append(row.text)
#for row in soup.find_all("ul")[2].findAll("li"):
#    TouristPlaceList.append(row.text)
#TouristPlaceList 2-14
for i in range(2,14):
    for row in soup.find_all("ul")[i].findAll("li"):
        TouristPlaceList.append(row.text)
TouristPlaceList = pd.DataFrame({"TouristPlace": TouristPlaceList})
TouristPlaceList.head()

,TouristPlace
0,Adlabs Imagica
1,Andheri Sports Complex
2,B.P.T. Ground
3,Bandra Kurla Complex Ground
4,Brabourne Stadium


In [9]:
TouristPlaceList.shape

(162, 1)

Get Geo Coordinates

In [10]:
def get_latlng(TouristPlace):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(TouristPlace))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(TouristPlace) for TouristPlace in TouristPlaceList["TouristPlace"].tolist() ]

In [12]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [13]:
# merge the coordinates into the original dataframe
TouristPlaceList['Latitude'] = df_coords['Latitude']
TouristPlaceList['Longitude'] = df_coords['Longitude']
# check the neighborhoods and the coordinates
print(TouristPlaceList.shape)
TouristPlaceList

(162, 3)


,TouristPlace,Latitude,Longitude
0,Adlabs Imagica,18.768290,73.282610
1,Andheri Sports Complex,19.128920,72.836290
2,B.P.T. Ground,19.017470,72.819200
3,Bandra Kurla Complex Ground,19.066840,72.861930
4,Brabourne Stadium,18.932580,72.823470
5,Colaba Woods,18.911520,72.819120
6,Cooperage Ground,18.924440,72.829580
7,Cross Maidan,18.939900,72.829850
8,Dadaji Kondadev Stadium,19.193170,72.980030
9,DY Patil Stadium,19.041350,73.027080


In [17]:
# save the DataFrame as CSV file
TouristPlaceList.to_csv("TouristPlaceList.csv", index=False)

#### Map of Mumbai

In [14]:
# get the coordinates of Kuala Lumpur
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [15]:
# create map of Toronto using latitude and longitude values
map_mi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, TouristPlace in zip(TouristPlaceList['Latitude'], TouristPlaceList['Longitude'], TouristPlaceList['TouristPlace']):
    label = '{}'.format(TouristPlace)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mi)  
    
map_mi

In [16]:
# save the map as HTML file
map_mi.save('map_mi_1.html')

Foursquare API to explore the neighborhoods of the Tourist places

Foursquare client ID

In [20]:
# @hidden_cell
CLIENT_ID = 'IQWXXT1WH2U52DVWG13LSQX22K344UH5HKDSVBJGRTI3YCCO' # your Foursquare ID
CLIENT_SECRET = 'GOKQJXYUPICIKIXGNVQYYA1W2DSLFCHGZKRQX105IVHIBDSX' # your Foursquare Secret
VERSION = '20200315' # Foursquare API version


In [22]:
radius = 3000
LIMIT = 100

venues = []

for lat, long, TouristPlace in zip(TouristPlaceList['Latitude'], TouristPlaceList['Longitude'], TouristPlaceList['TouristPlace']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            TouristPlace,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['TouristPlace', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(13957, 7)


,TouristPlace,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adlabs Imagica,18.76829,73.28261,Adlabs Imagica,18.767977,73.282027,Theme Park
1,Adlabs Imagica,18.76829,73.28261,Nitro,18.768260,73.281791,Theme Park Ride / Attraction
2,Adlabs Imagica,18.76829,73.28261,Adlabs Aquamagica,18.768281,73.282171,Water Park
3,Adlabs Imagica,18.76829,73.28261,Scream Machine,18.768881,73.281135,Theme Park Ride / Attraction
4,Adlabs Imagica,18.76829,73.28261,Mr. India,18.766420,73.280438,Indie Movie Theater


In [24]:
venues_df.groupby(["TouristPlace"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
TouristPlace,,,,,,
Capitol Cinema,8,8,8,8,8,8
National Gallery of Modern Art,100,100,100,100,100,100
Adlabs Imagica,14,14,14,14,14,14
Afghan Church,35,35,35,35,35,35
Aksa Beach,15,15,15,15,15,15
Andheri Sports Complex,100,100,100,100,100,100
Annapurna Studios,100,100,100,100,100,100
Antarang – Sex Health Information Art Gallery,100,100,100,100,100,100
B.P.T. Ground,55,55,55,55,55,55


Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 214 uniques categories.


In [26]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Theme Park', 'Theme Park Ride / Attraction', 'Water Park',
       'Indie Movie Theater', 'Indian Restaurant', 'American Restaurant',
       'Resort', 'Fast Food Restaurant', 'Breakfast Spot', 'Snack Place',
       'Pizza Place', 'Brewery', 'Multiplex', 'Falafel Restaurant',
       'Mediterranean Restaurant', 'Pub', 'Bakery', 'Café', 'Diner',
       'Mughlai Restaurant', 'Juice Bar', 'BBQ Joint', 'Lounge',
       "Women's Store", 'Ice Cream Shop', 'Coffee Shop', 'Bar',
       'Comfort Food Restaurant', 'Chinese Restaurant', 'Movie Theater',
       'Spa', 'Residential Building (Apartment / Condo)',
       'South Indian Restaurant', 'Jewelry Store', 'Gym',
       'Bengali Restaurant', 'Sandwich Place',
       'Vegetarian / Vegan Restaurant', 'Donut Shop', 'Dessert Shop',
       'Beach', 'Theater', 'Cupcake Shop', 'Italian Restaurant',
       'Cocktail Bar', 'Seafood Restaurant', 'Mexican Restaurant',
       'Hotel', 'French Restaurant', 'Scenic Lookout',
       'Athletics & Sports

In [35]:
# check if the results contain "Pizza Place"
"Pizza Place" in venues_df['VenueCategory'].unique()

True

In [33]:
VenueCategory_df=pd.DataFrame(venues_df['VenueCategory'].unique())
# save the DataFrame as CSV file
VenueCategory_df.to_csv("VenueCategory.csv", index=False)

Analyze Each Tourist Place

In [37]:
# one hot encoding
TouristPlaceList_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
TouristPlaceList_onehot['TouristPlace'] = venues_df['TouristPlace'] 

# move neighborhood column to the first column
fixed_columns = [TouristPlaceList_onehot.columns[-1]] + list(TouristPlaceList_onehot.columns[:-1])
TouristPlaceList_onehot = TouristPlaceList_onehot[fixed_columns]

print(TouristPlaceList_onehot.shape)
TouristPlaceList_onehot.head()

(13957, 215)


,TouristPlace,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Café,Caribbean Restaurant,Cave,Chaat Place,Cheese Shop,Chinese Restaurant,City,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Waterfront,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Adlabs Imagica,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Adlabs Imagica,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Adlabs Imagica,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [38]:
TouristPlaceList_grouped = TouristPlaceList_onehot.groupby(["TouristPlace"]).mean().reset_index()

print(TouristPlaceList_grouped.shape)
TouristPlaceList_grouped

(161, 215)


,TouristPlace,Afghan Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Café,Caribbean Restaurant,Cave,Chaat Place,Cheese Shop,Chinese Restaurant,City,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Punjabi Restaurant,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Track,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Water Park,Waterfront,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Capitol Cinema,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.125000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.250000,0